In [23]:
import numpy as np
from matplotlib import pyplot as plt
from data_loader import data_loader
from utils import params
from pydmd import DMD
from pydmd.plotter import plot_eigs
from sklearn.metrics import mean_squared_error as mse
import pandas as pd

In [24]:
def MAPE(y_true, y_pred):
    mask = y_true != 0
    y_true, y_pred = np.array(y_true)[mask], np.array(y_pred)[mask]
    if len(y_true) == 0:
        return np.nan
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [25]:
x_train, dx_train, x_val, dx_val = data_loader(params)
dmd = DMD(svd_rank=20)
dmd.fit(x_train.T)

In [26]:
measured_index = [i*params['embedding_dimension'] for i in range(params['partial_measurement'])]
x_val_measured = x_val[:, measured_index]
x_pred = dmd.predict(x_val.T).real.T
x_pred_2 = dmd.predict(x_pred.T).real.T

In [27]:
measured_index = [i*params['embedding_dimension'] for i in range(params['partial_measurement'])]
x_pred = x_pred[:, measured_index]
x_pred_2 = x_pred_2[:, measured_index]

In [28]:
np.save("Pred_Results/one_step_pred_DMD.npy", x_pred)
np.save("Pred_Results/x_val_DMD.npy", x_val_measured)

In [29]:
rmse_i = {}
for i in range(params['partial_measurement']):
    rmse_curr = np.sqrt(mse(100*x_pred[:-1, i], 100*x_val_measured[1:,i]))
    rmse_i[f'Station #{i+1}'] = rmse_curr

accuracy = pd.DataFrame(rmse_i, index=[1]).round(2)
accuracy.to_csv("Accuracy_Metric/dmd.csv")
accuracy

,Station #1,Station #2,Station #3,Station #4,Station #5,Station #6,Station #7,Station #8,Station #9,Station #10
1,33.73,26.76,28.13,37.17,44.0,27.7,30.48,67.93,24.89,32.09


In [30]:
mape_i = {}
for i in range(params['partial_measurement']):
    mape_curr = MAPE(100*x_val_measured[1:,i], 100*x_pred[:-1, i])
    mape_i[f'Station #{i+1}'] =mape_curr

accuracy_mape = pd.DataFrame(mape_i, index=[1]).round(2)
accuracy_mape.to_csv("Accuracy_Metric/dmd_MAPE.csv")
accuracy_mape

,Station #1,Station #2,Station #3,Station #4,Station #5,Station #6,Station #7,Station #8,Station #9,Station #10
1,6.64,6.81,7.14,8.43,12.75,11.64,12.08,21.81,12.73,34.55


In [31]:
rmse_i = {}
for i in range(params['partial_measurement']):
    rmse_curr = np.sqrt(mse(100*x_pred_2[:-2, i], 100*x_val_measured[2:,i]))
    rmse_i[f'Station #{i+1}'] = rmse_curr

accuracy = pd.DataFrame(rmse_i, index=[1]).round(2)
accuracy.to_csv("Accuracy_Metric/dmd_10mins.csv")
accuracy

,Station #1,Station #2,Station #3,Station #4,Station #5,Station #6,Station #7,Station #8,Station #9,Station #10
1,34.5,27.32,29.35,36.58,42.61,28.04,31.38,66.19,25.16,30.69


In [32]:
mape_i = {}
for i in range(params['partial_measurement']):
    mape_curr = MAPE(100*x_val_measured[2:,i], 100*x_pred_2[:-2, i])
    mape_i[f'Station #{i+1}'] =mape_curr

accuracy_mape = pd.DataFrame(mape_i, index=[1]).round(2)
accuracy_mape.to_csv("Accuracy_Metric/dmd_10mins_MAPE.csv")
accuracy_mape

,Station #1,Station #2,Station #3,Station #4,Station #5,Station #6,Station #7,Station #8,Station #9,Station #10
1,6.77,6.9,7.49,8.29,12.34,11.33,12.08,21.01,14.23,31.49


# Original DMD

In [33]:
from data_loader import data_loader_noH
x_train, dx_train, x_val, dx_val = data_loader_noH()

In [34]:
dmd = DMD(svd_rank=20)
dmd.fit(x_train.T)

In [35]:
x_pred = dmd.predict(x_val.T).real.T
x_pred_2 = dmd.predict(x_pred.T).real.T
x_pred_3 = dmd.predict(x_pred_2.T).real.T

In [36]:
rmse_i = {}
for i in range(params['partial_measurement']):
    rmse_curr = np.sqrt(mse(100*x_pred[:-1, i], 100*x_val[1:,i]))
    rmse_i[f'Station #{i+1}'] = rmse_curr

accuracy = pd.DataFrame(rmse_i, index=[1]).round(2)
accuracy.to_csv("Accuracy_Metric/dmd_ori.csv")
accuracy

,Station #1,Station #2,Station #3,Station #4,Station #5,Station #6,Station #7,Station #8,Station #9,Station #10
1,53.02,41.11,42.52,42.5,56.6,28.38,30.69,66.29,26.63,23.58


In [37]:
mape_i = {}
for i in range(params['partial_measurement']):
    mape_curr = MAPE(100*x_val[1:,i], 100*x_pred[:-1, i])
    mape_i[f'Station #{i+1}'] =mape_curr

accuracy_mape = pd.DataFrame(mape_i, index=[1]).round(2)
accuracy_mape.to_csv("Accuracy_Metric/dmd_ori_MAPE.csv")
accuracy_mape

,Station #1,Station #2,Station #3,Station #4,Station #5,Station #6,Station #7,Station #8,Station #9,Station #10
1,10.13,11.29,10.59,9.84,18.14,13.53,13.97,24.52,14.72,22.85


In [38]:
rmse_i = {}
for i in range(params['partial_measurement']):
    rmse_curr = np.sqrt(mse(100*x_pred_2[:-2, i], 100*x_val[2:,i]))
    rmse_i[f'Station #{i+1}'] = rmse_curr

accuracy = pd.DataFrame(rmse_i, index=[1]).round(2)
accuracy.to_csv("Accuracy_Metric/dmd_ori_10mins.csv")
accuracy

,Station #1,Station #2,Station #3,Station #4,Station #5,Station #6,Station #7,Station #8,Station #9,Station #10
1,66.67,51.77,53.06,51.34,68.99,32.71,35.31,84.19,32.04,25.78


In [39]:
mape_i = {}
for i in range(params['partial_measurement']):
    mape_curr = MAPE(100*x_val[2:,i], 100*x_pred_2[:-2, i])
    mape_i[f'Station #{i+1}'] =mape_curr

accuracy_mape = pd.DataFrame(mape_i, index=[1]).round(2)
accuracy_mape.to_csv("Accuracy_Metric/dmd_ori_10mins_MAPE.csv")
accuracy_mape

,Station #1,Station #2,Station #3,Station #4,Station #5,Station #6,Station #7,Station #8,Station #9,Station #10
1,12.65,14.96,14.0,11.84,23.27,16.93,17.3,33.02,17.93,26.32


In [40]:
np.save("Pred_Results/one_step_pred_DMDOri.npy", x_pred)
np.save("Pred_Results/x_val_DMDOri.npy", x_val)